In [1]:
import os
from sklearn.model_selection import train_test_split
from sklearn import svm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import joblib
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
d = dict()
class_labels = {
    0: '55',
    1: 'Green',
    2: 'Line',
    3: 'Red',
    4: 'Sheep',
    5: 'Stop'
}

# Define the path to your dataset
path = '/workspace/jetbot/notebooks/model/dataset'

# Image dimensions and batch size
img_rows, img_cols = 32, 32
batch_size = 11776

# Setup ImageDataGenerator
datagen = ImageDataGenerator()

# Generate data from the directories
data = datagen.flow_from_directory(
    path + '/myData',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical'
)

# Get the first batch of images and labels
images, labels = next(data)

Found 8223 images belonging to 6 classes.


In [3]:
import numpy as np

def center_crop(image, crop_size):
    img_height, img_width = image.shape[:2]
    crop_height, crop_width = crop_size

    # Calculate the center of the image
    center_y, center_x = img_height // 2, img_width // 2

    # Calculate the cropping box
    crop_y1 = max(0, center_y - crop_height // 2)
    crop_y2 = min(img_height, center_y + crop_height // 2)
    crop_x1 = max(0, center_x - crop_width // 2)
    crop_x2 = min(img_width, center_x + crop_width // 2)

    # Return the cropped image
    return image[crop_y1:crop_y2, crop_x1:crop_x2]

In [5]:
# Labels are one hot encoded
print(f"Data Shape   :{images.shape}\nLabels shape :{labels.shape}")

Data Shape   :(8223, 32, 32, 3)
Labels shape :(8223, 6)


In [6]:
# Flatten the images for SVM
images_flattened = images.reshape(images.shape[0], -1)
print(f"Flattened Shape: {images_flattened.shape}")

Flattened Shape: (8223, 3072)


In [ ]:
# Normalise the flattened images using StandardScaler
scaler = StandardScaler()
images_flattened_scaled = scaler.fit_transform(images_flattened)
print(f"Flattened Scaled Shape: {images_flattened_scaled.shape}")

In [7]:
# Convert one-hot encoded labels to integer labels
labels_int = np.argmax(labels, axis=1)  # Convert one-hot encoded labels to class indices
print(f"Labels Shape: {labels_int.shape}")

Labels Shape: (8223,)


In [8]:
# Split the data into training and testing sets (70% train, 30% test)
images_train, images_test, labels_train, labels_test = train_test_split(images_flattened, labels_int, test_size=0.3, random_state=0)
print("Finished")

Finished


In [9]:
model = svm.SVC(kernel='poly', C = 0.1, degree = 2, gamma = 0.01, verbose = 2, probability = True, class_weight='balanced')
model.fit(images_train, labels_train)

[LibSVM]

SVC(C=0.1, class_weight='balanced', degree=2, gamma=0.01, kernel='poly',
    probability=True, verbose=2)

In [10]:
labels_test_pred = model.predict(images_test)

In [11]:
# Metrics
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

test_accuracy = accuracy_score(labels_test, labels_test_pred)
test_precision = precision_score(labels_test, labels_test_pred, average='weighted')
test_recall = recall_score(labels_test, labels_test_pred, average='weighted')
test_f1 = f1_score(labels_test, labels_test_pred, average='weighted')

print(f'Accuracy: {test_accuracy:.8f}')
print(f'Precision: {test_precision:.8f}')
print(f'Recall: {test_recall:.8f}')
print(f'F1 Score: {test_f1:.8f}')

Accuracy: 1.00000000
Precision: 1.00000000
Recall: 1.00000000
F1 Score: 1.00000000


In [12]:
import joblib

# Save the model as a joblib file
joblib.dump(model, 'model.joblib')

# Save the test dataset (both images and labels) as joblib files
# joblib.dump(images_test, 'svm_images_test.joblib')
# joblib.dump(labels_test, 'svm_labels_test.joblib')

print("Model and test dataset have been saved successfully.")

Model and test dataset have been saved successfully.
